In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, LeakyReLU
from keras import backend
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error,explained_variance_score
import seaborn as sns

In [ ]:
call_df=pd.read_csv('/content/drive/MyDrive/black_scholes_option_pricing/complete_data/Call_with_svol_vix.csv')
put_df=pd.read_csv('/content/drive/MyDrive/black_scholes_option_pricing/complete_data/Put_with_svol_vix.csv')
call_df=call_df.set_index(['Date'])
put_df=put_df.set_index(['Date'])

In [ ]:
call_df.head()

,Strike Price,Close,Expiration,Adj Close,Price,bs_call,svol,Vix
Date,,,,,,,,
2013-12-31,6200,199.65,30,6304.0,8.82,188.621115,0.10855,15.08
2013-12-31,6300,131.00,30,6304.0,8.82,124.059028,0.10855,15.08
2013-12-31,6400,79.00,30,6304.0,8.82,75.195204,0.10855,15.08
2013-12-31,6500,41.35,30,6304.0,8.82,41.683668,0.10855,15.08
2013-12-31,6600,18.60,30,6304.0,8.82,21.015752,0.10855,15.08


In [ ]:
put_df.head()

,Strike Price,Close,Expiration,Adj Close,Price,bs_call,svol,Vix
Date,,,,,,,,
2013-12-31,5800,6.30,30,6304.0,8.82,0.780877,0.10855,15.08
2013-12-31,5900,10.30,30,6304.0,8.82,2.667110,0.10855,15.08
2013-12-31,6000,17.15,30,6304.0,8.82,7.656025,0.10855,15.08
2013-12-31,6100,28.80,30,6304.0,8.82,18.756159,0.10855,15.08
2013-12-31,6200,49.10,30,6304.0,8.82,39.837882,0.10855,15.08


In [ ]:
def regression(df):
  sns.scatterplot(x=df['Close'],y=df['bs_call'],hue=df['Expiration'])

In [ ]:
def split_data(df):
  call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(df[['Strike Price','Adj Close','Expiration','Price','svol','Vix']],df['Close'],test_size=0.2,random_state=42)
  return call_X_train, call_X_test, call_y_train, call_y_test

In [ ]:
def split_data_bs(df):
  call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(df[['Strike Price','Adj Close','Expiration','Price','svol','Vix']],df['bs_call'],test_size=0.2,random_state=42)
  return call_X_train, call_X_test, call_y_train, call_y_test

In [28]:
model_call=keras.models.load_model('/content/drive/MyDrive/black_scholes_option_pricing/MLP_CALL_MODEL.h5')
model_put=keras.models.load_model('/content/drive/MyDrive/black_scholes_option_pricing/MLP_PUT_MODEL.h5')


In [ ]:
def metrics(y_test,y_pred):
  x=np.array(y_test).reshape(-1,1)
  y=np.array(y_pred).reshape(-1,1)
  reg=LinearRegression().fit(x,y)
  mae= mean_absolute_error(x,y)
  mse=mean_squared_error(x,y)
  print("REG_SCORE - ",reg.score(x,y))
  print("Mean Absolute Error -",mae)
  print("Mean Squared Error -",mse)
  print("Explained_Variance_Score - ",explained_variance_score(x,y))
  

In [ ]:
def prediction(df,call):
  X_train, X_test, y_train, y_test=split_data(df)
  X_train_bs, X_test_bs, y_train_bs, y_test_bs=split_data_bs(df)
  if (call):
    y_pred = model_call.predict(X_test)
  else:
    y_pred = model_put.predict(X_test)
  print("********Metrics of MLP MODEL********")
  metrics(y_test,y_pred)
  print("")
  print("********Metrics of Black Scholes Model********")
  metrics(y_test,y_test_bs)
  print("")

In [ ]:
prediction(call_df,True)

********Metrics of MLP MODEL********
REG_SCORE -  0.9606508007614629
Mean Absolute Error - 28.96619784497376
Mean Squared Error - 6496.118640920582
Explained_Variance_Score -  0.9602905881630084

********Metrics of Black Scholes Model********
REG_SCORE -  0.9494898762123269
Mean Absolute Error - 38.45105511123691
Mean Squared Error - 8722.47606557437
Explained_Variance_Score -  0.9468373613438806



In [ ]:
prediction(put_df,False)

********Metrics of MLP MODEL********
REG_SCORE -  0.9717387079575286
Mean Absolute Error - 27.060572766788162
Mean Squared Error - 6305.561960564497
Explained_Variance_Score -  0.9714242019205003

********Metrics of Black Scholes Model********
REG_SCORE -  0.9590950800387738
Mean Absolute Error - 46.866418666752175
Mean Squared Error - 9421.919792591612
Explained_Variance_Score -  0.9579455057191564

